In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<p style="font-family: 'Amiri'; font-size: 3rem; color: white; text-align: center; margin: 0; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3); background-color: #2b8a3e; padding: 20px; border-radius: 20px; border: 7px solid #f1c40f; width:95%">
 ｡CIBMTR - Equity in Post-HCT Survival Prediction˚
</p>
>


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
import plotly.express as px
from IPython.display import display, HTML
import warnings

In [ ]:
from colorama import Fore, Style
from catboost import CatBoostClassifier
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, KFold
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score, roc_auc_score
from lightgbm import LGBMClassifier

In [ ]:
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')

### Importing dataset

In [ ]:
train = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/train.csv')
test = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
train_solution=train[['ID','efs','efs_time','race_group']].copy()
solution = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv')

In [ ]:
from lifelines import KaplanMeierFitter

def transform_survival_probability(df, time_col='efs_time', event_col='efs'):
    """
    Transforms the survival probabilities using Kaplan-Meier estimator.
    
    Parameters:
        df (pd.DataFrame): The dataframe containing the survival data.
        time_col (str): The name of the column containing time data.
        event_col (str): The name of the column containing event/censoring information.
    
    Returns:
        pd.Series: A series of survival probabilities.
    """
    kmf = KaplanMeierFitter()
    
    # Fit Kaplan-Meier estimator
    kmf.fit(df[time_col], event_observed=df[event_col])
    
    # Get survival probabilities at each time point
    survival_probabilities = kmf.survival_function_at_times(df[time_col]).values.flatten()
    
    return survival_probabilities

# Apply transformation to the training dataset
train["target"] = transform_survival_probability(train)

# Columns to drop
drop_cols = ["ID", 'efs', 'efs_time']

# Drop the columns from both training and test datasets
train.drop(columns=drop_cols, inplace=True, errors='ignore')
test.drop(columns=drop_cols, inplace=True, errors='ignore')


## Peak data

In [ ]:
import pandas as pd

# Function to print styled heading
def print_heading(text, color="yellow"):
    print(f"\033[1;33;40m{'='*50}")
    print(f"{text.center(50)}")
    print(f"{'='*50}\033[0m")

# Helper function for error messages
def print_error(message):
    print(f"\033[1;31;40m{'='*50}")
    print(f"Error: {message}")
    print(f"{'='*50}\033[0m")

# Function to display dataset summary
def display_dataset_summary(dataset, dataset_name="Dataset"):
    try:
        print_heading(f"{dataset_name} Shape")
        print(f"{dataset_name}: {dataset.shape[0]} rows, {dataset.shape[1]} columns\n")
        
        # Dataset Summary
        print_heading(f"{dataset_name} Summary")
        print(dataset.describe())
        
        # Null values
        print_heading(f"Null Values in {dataset_name}")
        null_count = dataset.isnull().sum()
        null_percentage = (null_count / len(dataset)) * 100
        null_summary = pd.DataFrame({
            "Null Count": null_count[null_count > 0],
            "Null Percentage (%)": null_percentage[null_percentage > 0]
        })
        if null_count.sum() == 0:
            print(f"No null values in the {dataset_name}.")
        else:
            print(null_summary)
        
        # Duplicate rows
        print_heading(f"Duplicate Values in {dataset_name}")
        duplicates = dataset.duplicated().sum()
        print(f"{dataset_name}: {duplicates} duplicate rows\n")

    except Exception as e:
        print_error(str(e))

# Function to analyze both train and test datasets
def analyze_datasets(train_dataset, test_dataset, n_top=5):
    try:
        # Display top rows of train dataset
        print_heading(f"Top {n_top} Rows of Training Dataset")
        print(train_dataset.head(n_top))
        
        # Display top rows of test dataset
        print_heading(f"Top {n_top} Rows of Test Dataset")
        print(test_dataset.head(n_top))
        
        # Display dataset summaries
        display_dataset_summary(train_dataset, "Training Dataset")
        display_dataset_summary(test_dataset, "Test Dataset")

    except Exception as e:
        print_error(str(e))

# Function to print unique values in columns
def print_unique_values(dataset):
    try:
        print_heading("Unique Values in Dataset")
        
        for column in dataset.columns:
            unique_values = dataset[column].unique()[:7]  # Limiting to first 7 unique values
            data_type = dataset[column].dtype
            print(f"Column: {column}, Data Type: {data_type}, Unique Values: {', '.join(map(str, unique_values))}")
    
    except Exception as e:
        print_error(str(e))

In [ ]:
analyze_datasets(train, test)  # Correct function call
print_unique_values(train) 

<p style="font-family: 'Amiri'; font-size: 3rem; color: white; text-align: center; margin: 0; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3); background-color: #34495e; padding: 20px; border-radius: 20px; border: 7px solid #e74c3c; width:95%">
   Data Visualization˚
</p>


In [ ]:
def replace_nulls_with_default(df, float_default=0.0, object_default="Unknown"):
    """
    Replace null values in a DataFrame with default values based on data type.

    Parameters:
        df (pd.DataFrame): The DataFrame to process.
        float_default (float, optional): Default value for float columns. Default is 0.0.
        object_default (str, optional): Default value for object (string) columns. Default is "Unknown".
    
    Returns:
        pd.DataFrame: DataFrame with nulls replaced.
    """
    # Replace nulls in float columns
    df.update(df.select_dtypes(include=['float']).fillna(float_default))
    # Replace nulls in object columns
    df.update(df.select_dtypes(include=['object']).fillna(object_default))
    return df


In [ ]:
train = replace_nulls_with_default(train, float_default=-1.0, object_default="Missing")
test = replace_nulls_with_default(test, float_default=-1.0, object_default="Missing")


In [ ]:
def convert_object_to_category(df):
    """
    Converts all columns with object data type to category data type in a DataFrame.

    Parameters:
        df (pd.DataFrame): The DataFrame to process.
    
    Returns:
        pd.DataFrame: The updated DataFrame with object columns converted to category.
    """
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].astype('category')
    return df

In [ ]:
train = convert_object_to_category(train)
test = convert_object_to_category(test)

In [ ]:
def print_value_counts_less_than(df, threshold=50):
    for col in df.columns:
        value_counts = df[col].value_counts()
        if len(value_counts) < threshold:
            print(f"Value counts for column '{col}':")
            for val, count in value_counts.items():
                print(f"    {val}: {count}")
            print()

print_value_counts_less_than(train, threshold=30)

In [ ]:
import matplotlib.pyplot as plt

def plot_low_value_count_columns_pie(df):
    # Identify columns with less than 20 unique values
    low_value_count_columns = [col for col in df.columns if df[col].nunique() < 20]
    
    # Custom color palette for the pie chart
    custom_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
    
    # Iterate through each of the identified columns
    for col in low_value_count_columns:
        # Get the value counts of the column
        value_counts = df[col].value_counts()
        
        # Plotting a pie chart with annotations for each slice
        plt.figure(figsize=(8, 8))
        wedges, texts, autotexts = plt.pie(value_counts, 
                                           labels=value_counts.index, 
                                           autopct='%1.1f%%', 
                                           startangle=90, 
                                           colors=custom_colors[:len(value_counts)])
        
        # Adding a title and customizing the plot
        plt.title(f'Distribution of {col}', fontsize=16, fontweight='bold', color='darkblue')
        
        # Customize annotations (the text inside the slices)
        for text in texts:
            text.set(fontsize=12, color='black', fontweight='bold')
        
        for autotext in autotexts:
            autotext.set(fontsize=10, color='white', fontweight='bold')
        
        # Show plot
        plt.show()

# Example usage
plot_low_value_count_columns_pie(train)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

def separate_int_columns(df):
    return df.select_dtypes(include=['int'])

numerical_columns = [col for col in train.select_dtypes(include=['number']).columns if col != 'Target']

# Correlation Heatmap
correlation_matrix = train[numerical_columns].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, linewidths=0.5, fmt='.2f', cbar=True)
plt.title('Correlation Heatmap of Numerical Columns', fontsize=16, fontweight='bold', color='darkblue')
plt.show()

# Individual Histograms for Each Numerical Column
fig, axes = plt.subplots(nrows=len(numerical_columns), ncols=1, figsize=(10, len(numerical_columns)*4))
axes = axes.flatten()

for i, col in enumerate(numerical_columns):
    sns.histplot(train[col], kde=True, ax=axes[i], color='skyblue', bins=20)
    axes[i].set_title(f'{col} Distribution', fontsize=14, fontweight='bold', color='darkblue')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()


<p style="font-family: 'Amiri'; font-size: 3rem; color: white; text-align: center; margin: 0; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3); background-color: #34495e; padding: 20px; border-radius: 20px; border: 7px solid #e74c3c; width:95%">
   ML Algorithms 
</p>

In [ ]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import StratifiedKFold

# Custom function to compute the Stratified Concordance Index (C-index)
def stratified_c_index(y_true, y_pred, groups):
    """
    Compute the Stratified Concordance Index (C-index) for the predicted values.
    
    Args:
        y_true (array): True values of the target variable.
        y_pred (array): Predicted values of the target variable.
        groups (array): Grouping variable for stratified computation (e.g., race groups).
        
    Returns:
        float: Stratified C-index score.
    """
    unique_groups = np.unique(groups)
    c_indices = []

    for group in unique_groups:
        mask = groups == group
        if sum(mask) > 1:  # Only consider groups with more than one member
            y_true_group = y_true[mask]
            y_pred_group = y_pred[mask]
            concordant = 0
            permissible = 0

            for i in range(len(y_true_group)):
                for j in range(i + 1, len(y_true_group)):
                    if y_true_group[i] != y_true_group[j]:
                        permissible += 1
                        if (y_pred_group[i] > y_pred_group[j] and y_true_group[i] > y_true_group[j]) or \
                           (y_pred_group[i] < y_pred_group[j] and y_true_group[i] < y_true_group[j]):
                            concordant += 1

            c_indices.append(concordant / permissible if permissible > 0 else 0)

    c_indices = np.array(c_indices)
    return np.mean(c_indices) - np.std(c_indices)

# 1. Prepare the dataset (assuming df_tr is predefined)
X = train.drop(columns=['target'], axis=1)  # Features
y = train['target']  # Target variable
race_groups = train['race_group']  # Grouping variable for stratified C-index

# 2. Identify categorical features for CatBoost
cat_features = list(X.select_dtypes(include=['object', 'category']).columns)

# 3. Initialize K-Fold Cross Validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
final_predictions = np.zeros(len(X))  # To store final predictions

# 4. Train and validate the model using Stratified K-Fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X, race_groups)):
    print(f"Training on Fold {fold + 1}")
    
    # Split data into train and validation sets
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    race_val = race_groups.iloc[val_idx]

    # Prepare Pool datasets for CatBoost
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    val_pool = Pool(X_val, y_val, cat_features=cat_features)

    # Initialize CatBoostRegressor model
    model = CatBoostRegressor(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        l2_leaf_reg=3,
        loss_function='RMSE',
        random_seed=42,
        verbose=100
    )

    # Train the model with early stopping
    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=50)

    # Predict and compute the Stratified C-index
    y_val_pred = model.predict(X_val)
    fold_score = stratified_c_index(y_val.values, y_val_pred, race_val.values)
    print(f"Stratified C-Index for Fold {fold + 1}: {fold_score:.4f}")

    # Store predictions
    final_predictions[val_idx] = y_val_pred

# 5. Compute the Overall Stratified C-index
overall_score = stratified_c_index(y.values, final_predictions, race_groups.values)
print(f"Overall Stratified C-Index: {overall_score:.4f}")


In [ ]:
pred = model.predict(test)


print(pred)


In [ ]:
solution.head()

In [ ]:
solution['prediction'] = pred
solution.to_csv('submission.csv', index=False)

In [ ]:
solution.head()

## Thank you